<a href="https://colab.research.google.com/github/MariaKCodes/OpenMScThesisInNeuroPixelProbeDataAnalysis/blob/main/Spikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ONE-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 4.6 MB/s 
     |████████████████████████████████| 132 kB 69.0 MB/s 
     |████████████████████████████████| 61 kB 553 kB/s 
     |████████████████████████████████| 66 kB 5.2 MB/s 
     |████████████████████████████████| 41 kB 378 kB/s 
     |████████████████████████████████| 79 kB 9.1 MB/s 
     |████████████████████████████████| 9.2 MB 66.1 MB/s 
     |████████████████████████████████| 127 kB 50.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.12.0
    Uninstalling importlib-metadata-4.12.0:
      Successfully uninstalled importlib-metadata-4.12.0
ERROR: pip's dependency resolver does not currently take into account all the package

In [4]:
from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org', password='international', silent=True)
eids, info = one.search(dataset='probes.description', task_protocol='_iblrig_tasks_ephysChoiceWorld6.4.2', details=True)


In [6]:
eid = eids[0]  
probe_insertions = one.load_dataset(eid, 'probes.description')

from pprint import pprint
print(f'N probes = {len(probe_insertions)}')
pprint(probe_insertions[0])
pprint(probe_insertions[1])

N probes = 2
{'label': 'probe00',
 'model': '3B2',
 'raw_file_name': 'D:/iblrig_data/Subjects/UCLA037/2022-02-02/001/raw_ephys_data/probe00/_spikeglx_ephysData_g0/_spikeglx_ephysData_g0_imec0/_spikeglx_ephysData_g0_t0.imec0.ap.bin',
 'serial': 19051010091}
{'label': 'probe01',
 'model': '3B2',
 'raw_file_name': 'D:/iblrig_data/Subjects/UCLA037/2022-02-02/001/raw_ephys_data/probe01/_spikeglx_ephysData_g0/_spikeglx_ephysData_g0_imec1/_spikeglx_ephysData_g0_t0.imec1.ap.bin',
 'serial': 19051010152}


In [ ]:
probe_label = probe_insertions[0]['label']
one.list_datasets(eid, collection=f'alf/{probe_label}')

['alf/probe00/electrodeSites.brainLocationIds_ccf_2017.npy',
 'alf/probe00/electrodeSites.localCoordinates.npy',
 'alf/probe00/electrodeSites.mlapdv.npy']

In [7]:
eid

'2bdf206a-820f-402f-920a-9e86cd5388a4'

In [11]:
!pip install brainbox.io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement brainbox.io (from versions: none)
ERROR: No matching distribution found for brainbox.io


In [8]:
from one.api import ONE
from brainbox.io.one import SpikeSortingLoader
from ibllib.atlas import AllenAtlas

one = ONE()
ba = AllenAtlas()
pid = eid

sl = SpikeSortingLoader(pid=pid, one=one, atlas=ba)
spikes, clusters, channels = sl.load_spike_sorting()
clusters = sl.merge_clusters(spikes, clusters, channels)

ModuleNotFoundError: ignored

In [42]:
spikes = one.load_object(eid, 'spikes',
                         collection='alf/probe00/pykilosort', attribute=['times', 'channels', 'clusters'])
spikes

{'clusters': array([ 90,  20, 172, ..., 117, 135, 343], dtype=uint32),
 'times': array([1.32594040e-02, 1.38260698e-02, 1.38927363e-02, ...,
        5.10921711e+03, 5.10921714e+03, 5.10921811e+03])}

In [43]:
import numpy as np
cl=spikes['clusters']
np.unique(cl)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [32]:
def bin_spikes(spike_times,spike_clusters, bin_size = 10):

    # Using clusters._phy_annotation.npy obtain valid clusters (i.e. >= 2)
    # valid_clusters_idx = np.array(np.where(clusters_annotation>=2))[0]

    spike_time_cells = np.empty(np.unique(spike_clusters).shape[0], dtype=object) # Initalise empty object
    for i in (np.arange(len(np.unique(spike_clusters)))):
      # Create a spike time arrays, where each array in the array is a spike time of a cell
      spike_time_cells[i] = spike_times[(np.where(spike_clusters == i)[0])]

    # Bin spike times into 10ms intervals
    spike_time_binned = np.empty(len(np.unique(spike_clusters)), dtype=object) # Initalise empty object
    sum_spikes = np.empty(len(np.unique(spike_clusters)), dtype=object) # Initalise empty object

    for cell_num in np.arange(len(spike_time_cells)):
        spike_time_hist = np.histogram(spike_time_cells[cell_num],bins = np.arange(0,np.floor(spike_time_cells[cell_num][-1]),bin_size))
        spike_time_binned[cell_num] = spike_time_hist[0]
        sum_spikes[cell_num] = np.sum(spike_time_binned[cell_num])

    cell_spikes_max = np.argmax(sum_spikes) # cell with the maximum number of spikes for plotting purposes

    # Spike_time_binned returns binned spikes sorted into cells
    # Spike_time_cells returns UNbinned spikes sorted into cells
    # cell_spikes_max returns a single cell index that has the max number of spikes (i.e most active cell)
    return spike_time_binned, spike_time_cells, cell_spikes_max

In [23]:
spike_clusters=spikes['clusters']
spike_times=spikes['times']

In [33]:
spike_time_binned, spike_time_cells, cell_spikes_max=bin_spikes(spike_times,spike_clusters, bin_size = 10)

In [35]:
valid=np.unique(spike_clusters[spike_clusters>=2])
print(valid.shape)

(501,)


In [25]:
import matplotlib.pyplot as plt

print(spike_time_binned.shape)

(503,)


In [26]:
brain_locs=one.load_dataset(eid, 'electrodeSites.brainLocationIds_ccf_2017.npy')

/root/Downloads/ONE/openalyx.internationalbrainlab.org/churchlandlab_ucla/Subjects/UCLA037/2022-02-02/001/alf/probe00/electrodeSites.brainLocationIds_ccf_2017.npy: 100%|██████████| 3.20k/3.20k [00:00<00:00, 12.9kB/s]


In [27]:
brain_locs.shape

(384,)

In [36]:
probe_label = probe_insertions[0]['label']
one.list_datasets(eid, collection=f'alf/{probe_label}')

['alf/probe00/electrodeSites.brainLocationIds_ccf_2017.npy',
 'alf/probe00/electrodeSites.localCoordinates.npy',
 'alf/probe00/electrodeSites.mlapdv.npy']

In [41]:
brain_locs=one.load_dataset(eid, 'alf/probe00/electrodeSites.brainLocationIds_ccf_2017.npy')
brain_locs.shape

(384,)

In [44]:
!pip install ibllib
!pip install brainbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 13.6 MB/s 
     |████████████████████████████████| 96 kB 5.1 MB/s 
     |████████████████████████████████| 15.7 MB 59.0 MB/s 
     |████████████████████████████████| 8.8 MB 46.0 MB/s 
     |████████████████████████████████| 131 kB 51.5 MB/s 
     |████████████████████████████████| 4.0 MB 52.8 MB/s 
     |████████████████████████████████| 120 kB 52.7 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 346 kB 45.4 MB/s 
     |████████████████████████████████| 9.8 MB 51.3 MB/s 
     |████████████████████████████████| 423 kB 68.5 MB/s 
     |████████████████████████████████| 272 kB 69.8 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 80 kB 10.0 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████|

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement brainbox (from versions: none)
ERROR: No matching distribution found for brainbox
